In [10]:
import numpy as np
from pc_kriging import PC_Kriging
import matplotlib.pyplot as plt
from scipy.stats import norm
from doepy import build
from scipy import optimize
import pandas as pd
from numpy import genfromtxt
import pickle

from scipy.special import eval_legendre, eval_hermitenorm
import scipy.special

# adaptive learning - expected feasiability function --------------------------------------------------------------

def EFF(u,v,z):
    zl=-2*v
    zh=2*v
    return ((u-z)*( 2*norm.cdf((z-u)/v) - norm.cdf((zl-u)/v) - norm.cdf((zh-u)/v)) 
           -(v)*( 2*norm.pdf((z-u)/v) - norm.pdf((zl-u)/v) - norm.pdf((zh-u)/v))  
           +(2*v)*(norm.cdf((zh-u)/v) - norm.cdf((zl-u)/v)))

def U_function(u, v):
    return np.abs(u)/v

def LinearNorm(x,oldmin,oldmax,newmin,newmax):    # scaling linearly X to new domain limits
    return newmin + ((x-oldmin)*(newmax-newmin)/(oldmax-oldmin))


def VoronoiCell(x,xn):   #given x [single value] return the index of the closest xn [array]
    dist=PCK1.distance(x.reshape(1,-1),xn)
    return np.argmin(dist)

In [11]:
# #ground truth function----------------------------------------------------
# https://rprepo.readthedocs.io/en/latest/reliability_problems.html#rp201

def gfun_53(x):
    """Performance function for reliability problem 53.

    Parameters
    ----------
        x : numpy.array of float(s)
            Values of independent variables: columns are the different parameters/random variables (x1, x2,...xn) and rows are different parameter/random variables sets for different calls.

    Returns
    -------
        g_val_sys : numpy.array of float(s)
            Performance function value for the system.
        g_val_comp : numpy.array of float(s)
            Performance function value for each component.
        msg : str
            Accompanying diagnostic message, e.g. warning.
    """
#     import numpy as np
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(5*x[:, 0]/2) + 2 - (x[:, 0]**2 + 4)*(x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

In [12]:
config = {"pol_type": ['hermite', 'hermite']}   #design variables following normal distribution
PCK1 = PC_Kriging(config)
PCK_loo = PC_Kriging(config)    # for LOOCV with same 'config' as specified in the original model

In [13]:
# objective function to optimize length scale --------------------------------------------------------
def L_Object (l):
    v = 5/2
    N = len(xn)
    R = PCK1.matern(xn , xn, l, v)
    detR = np.linalg.det(R)
    
    modelpar2 = PCK1.train(xn, yn, p, np.array([l,v]))    # returns B, sig2, InfoMatrix(phi) , PolyIndices(alpha)
    ### ------------------Theta_ by UQLab User Manual PCK(C. Lataniotis, D. Wicaksono, S. Marelli, B. Sudret)------------------------------
    sig2 = modelpar2[1].reshape(-1)
    # return 0.5*(np.log(detR)+ N*np.log(2*np.pi*sig2)+ N)

    ### ------------------Theta_ by MLE PCK(Schobi,Sudret,Wiart)------------------------------
    FB = PCK1.InfoMat @ modelpar2[0]
    ins = (yn-FB).reshape(-1)
    R_1 = np.linalg.inv(R)
    return ((ins.T) @ R_1 @ ins) * (1/N) * (detR**(1/N))

## Initial training 

In [14]:
function = gfun_53   #choose the ground truth function
dim = 2       # dimensionality

ntest = 5000  # test points

x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
x2mean, x2sigma = 2.5 , 1.0  # normal distribution 
    
# TEST POINTS -------------------------------------------------
XR = np.zeros((int(ntest), dim))   #normalized test points
XN = np.zeros((int(ntest), dim))  #scaled test points
YN = np.zeros(int(ntest)) 
#variable 1 ---------------------------------------------------
XN[:,0] = np.random.normal(0,1,ntest)  
XR[:,0] = PCK1.scalehermite(XN[:,0], x1mean, x1sigma)  
#variable 2 ---------------------------------------------------
XN[:,1] = np.random.normal(0,1,ntest)  
XR[:,1] = PCK1.scalehermite(XN[:,1], x2mean, x2sigma)  

YN = function(XR)[0]

# Active training - U - 1 LS

In [16]:
number_experiments = 15
number_active_points = 40

results_file = '_AT_U_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '------------------------------------------------------------' )
    
    ActiveTrain = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #initial design of experiments (LHS) --------------------------------

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    mean_loo = np.zeros(len(xn))
    var_loo = np.zeros(len(xn))



    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn = function(xr)[0]

    for points in range(number_active_points):

        ModelName = 'PCK' + str(len(xn))

        ModelName = PC_Kriging(config)

        # Selecting the best model (length and order) 
        # kernel hyperparameters------------------------------
        v = 5/2
        
        #truncation term-------------------------------------
        p_max = 6  #for each variable → same truncation , degree of expansion

        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        eloo_results = np.zeros(p_max-1)

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            results = dict()
            bounds = [(lmin, lmax)]
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName.train (xn, yn, p, theta) 
            mean0, var0 = ModelName.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  

        # ---------------------------------------------------
        ymc = function(MCinputs)[0]  
        fail_samples = np.sum(ymc < 0 )
        Pf_ref = fail_samples/MCS_samples

        meanMC, varMC = ModelName.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO = np.sum(np.asarray(meanMC) < 0 )
        fail_prob_SUMO = fail_samples_SUMO / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO ) / (fail_prob_SUMO * MCS_samples) )


        print('Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf', fail_prob_SUMO , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain[str(len(xn))+'points'] = ModelName , fail_prob_SUMO , cov_pf , np.min(eloo_results), np.min(mse_results)

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), varMC.reshape(-1))
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)]).reshape(-1,2)
        yn = function(xr)[0] #Only one point needs to be evaluated here

        print('number of training points: ', len(xn))
        
    filename = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'
    pickle.dump(ActiveTrain, open(filename, 'wb'))

Experiment:  1 ------------------------------------------------------------


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Degree 2 e_LOO 0.00021014898237079835 Pf 0.09206 CoV 0.00990
number of training points:  11
Degree 1 e_LOO 0.0009081113509667807 Pf 0.06708 CoV 0.01180
number of training points:  12
Experiment:  2 ------------------------------------------------------------
Degree 1 e_LOO 0.0025942726030455804 Pf 0.02479 CoV 0.01980
number of training points:  11
Degree 2 e_LOO 0.0002451406146415132 Pf 0.11081 CoV 0.00900
number of training points:  12


# Active training - EFF - 1 LS

In [288]:
number_experiments = 15
number_active_points = 40

results_file = '_AT_EFF_' + str(number_active_points)    # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '------------------------------------------------------------' )
    
    ActiveTrain = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #initial design of experiments (LHS) --------------------------------

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    mean_loo = np.zeros(len(xn))
    var_loo = np.zeros(len(xn))

    x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
    x2mean, x2sigma = 2.5 , 1.0  # normal distribution 

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn = function(xr)[0]

    for points in range(number_active_points):

        ModelName = 'PCK' + str(len(xn))

        ModelName = PC_Kriging(config)

        # Selecting the best model (length and order) 
        # kernel hyperparameters------------------------------
        v = 5/2
        #n = 11 
        #truncation term-------------------------------------
        p_max = 5  #for each variable → same truncation , degree of expansion

        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        eloo_results = np.zeros(p_max-1)

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            results = dict()
            bounds = [(lmin, lmax)]
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = PCK1.train (xn, yn, p, theta) 
            mean0, var0 = PCK1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  

        # ---------------------------------------------------
        ymc = function(MCinputs)[0]  
        fail_samples = np.sum(ymc < 0 )
        Pf_ref = fail_samples/MCS_samples

        meanMC, varMC = ModelName.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO = np.sum(np.asarray(meanMC) < 0 )
        fail_prob_SUMO = fail_samples_SUMO / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO ) / (fail_prob_SUMO * MCS_samples) )


        print('Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf', fail_prob_SUMO , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain[str(len(xn))+'points'] = ModelName , fail_prob_SUMO , cov_pf , np.min(eloo_results), np.min(mse_results)

        ### Evaluating new points
        eff1 = EFF(meanMC.reshape(-1),varMC.reshape(-1) , 0 )  #(mean, variance, target)  
        
        xr = np.append(xr, MCinputs[np.argmax(eff1)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff1)]).reshape(-1,2)
        yn = function(xr)[0] #Only one point needs to be evaluated here

        print('number of training points: ', len(xn))
    
    filename = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'
    pickle.dump(ActiveTrain, open(filename, 'wb'))    #saving dictionary after each experiment

Experiment:  1 ------------------------------------------------------------


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Degree 1 e_LOO 0.002328474726899101 Pf 0.07052 CoV 0.01150
number of training points:  11
Degree 3 e_LOO 0.005089218922686236 Pf 0.1078 CoV 0.00910
number of training points:  12
Degree 3 e_LOO 0.00791262397336823 Pf 0.10281 CoV 0.00930
number of training points:  13
Degree 1 e_LOO 0.0060862443055089335 Pf 0.014 CoV 0.02650
number of training points:  14
Degree 1 e_LOO 0.005670376101977913 Pf 0.008 CoV 0.03520
number of training points:  15
Degree 2 e_LOO 0.004662697633464333 Pf 0.01109 CoV 0.02990
number of training points:  16
Degree 1 e_LOO 9.223222008663385e-05 Pf 0.00269 CoV 0.06090
number of training points:  17
Degree 1 e_LOO 0.000507940031832551 Pf 0.02625 CoV 0.01930
number of training points:  18
Degree 1 e_LOO 0.0013870698837998132 Pf 0.02106 CoV 0.02160
number of training points:  19
Degree 1 e_LOO 0.0004905043116430428 Pf 0.02861 CoV 0.01840
number of training points:  20
Degree 1 e_LOO 0.0010054633426089354 Pf 0.02271 CoV 0.02070
number of training points:  21
Degree 1 e_

Degree 3 e_LOO 0.0006675881519275072 Pf 0.01162 CoV 0.02920
number of training points:  40
Experiment:  4 ------------------------------------------------------------
Degree 1 e_LOO 0.002745609610418422 Pf 0.08186 CoV 0.01060
number of training points:  11
Degree 1 e_LOO 0.02391714683425778 Pf 0.13491 CoV 0.00800
number of training points:  12
Degree 1 e_LOO 3.1915697153927166e-05 Pf 0.12098 CoV 0.00850
number of training points:  13
Degree 2 e_LOO 0.0018808195626974762 Pf 0.10234 CoV 0.00940
number of training points:  14
Degree 1 e_LOO 0.021236584257827658 Pf 0.08904 CoV 0.01010
number of training points:  15
Degree 1 e_LOO 0.025842721652379403 Pf 0.08622 CoV 0.01030
number of training points:  16
Degree 2 e_LOO 0.0002891820138581591 Pf 0.07211 CoV 0.01130
number of training points:  17
Degree 2 e_LOO 0.0004181734627905606 Pf 0.07041 CoV 0.01150
number of training points:  18
Degree 2 e_LOO 0.0022761414375471133 Pf 0.07647 CoV 0.01100
number of training points:  19
Degree 2 e_LOO 0.0

Degree 4 e_LOO 0.0005536759549196768 Pf 0.01074 CoV 0.03030
number of training points:  38
Degree 4 e_LOO 0.0003786520424274373 Pf 0.01119 CoV 0.02970
number of training points:  39
Degree 4 e_LOO 0.0020770511994679663 Pf 0.01228 CoV 0.02840
number of training points:  40
Experiment:  7 ------------------------------------------------------------
Degree 1 e_LOO 0.000943683326169192 Pf 0.0891 CoV 0.01010
number of training points:  11
Degree 1 e_LOO 0.00020077232417652907 Pf 0.1107 CoV 0.00900
number of training points:  12
Degree 1 e_LOO 0.06638977653507552 Pf 0.03874 CoV 0.01580
number of training points:  13
Degree 1 e_LOO 0.0040128779242518 Pf 0.00893 CoV 0.03330
number of training points:  14
Degree 2 e_LOO 7.107138382701552e-05 Pf 0.04287 CoV 0.01490
number of training points:  15
Degree 1 e_LOO 0.01443431400152952 Pf 0.00346 CoV 0.05370
number of training points:  16
Degree 2 e_LOO 0.014397086424363372 Pf 0.0592 CoV 0.01260
number of training points:  17
Degree 2 e_LOO 0.00772887

Degree 4 e_LOO 4.495884166178114e-05 Pf 0.01122 CoV 0.02970
number of training points:  37
Degree 2 e_LOO 8.480329429031133e-08 Pf 0.01554 CoV 0.02520
number of training points:  38
Degree 2 e_LOO 6.096754268230917e-06 Pf 0.01463 CoV 0.02600
number of training points:  39
Degree 2 e_LOO 9.861932775183201e-05 Pf 0.01483 CoV 0.02580
number of training points:  40
Experiment:  10 ------------------------------------------------------------
Degree 1 e_LOO 0.014568640581209625 Pf 0.04735 CoV 0.01420
number of training points:  11
Degree 3 e_LOO 0.006026680171490102 Pf 0.16626 CoV 0.00710
number of training points:  12
Degree 2 e_LOO 0.00014760394989697304 Pf 0.00359 CoV 0.05270
number of training points:  13
Degree 1 e_LOO 0.03174255806563008 Pf 4e-05 CoV 0.50000
number of training points:  14


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:127: RuntimeWarning: divide by zero encountered in double_scalars


Degree 1 e_LOO 0.045632024508420944 Pf 0.0 CoV inf
number of training points:  15
Degree 1 e_LOO 0.0029736503539422744 Pf 0.00128 CoV 0.08830
number of training points:  16
Degree 1 e_LOO 0.004716338978831698 Pf 0.00187 CoV 0.07310
number of training points:  17
Degree 1 e_LOO 0.004561018452827755 Pf 0.00221 CoV 0.06720
number of training points:  18
Degree 1 e_LOO 4.729196663394916e-05 Pf 0.0117 CoV 0.02910
number of training points:  19
Degree 3 e_LOO 6.271977267744197e-06 Pf 0.06985 CoV 0.01150
number of training points:  20
Degree 2 e_LOO 0.00016284180891657052 Pf 0.01381 CoV 0.02670
number of training points:  21
Degree 2 e_LOO 0.00135107300382824 Pf 0.0217 CoV 0.02120
number of training points:  22
Degree 1 e_LOO 0.0019494896710939898 Pf 0.02114 CoV 0.02150
number of training points:  23
Degree 2 e_LOO 0.0019295026657366152 Pf 0.01998 CoV 0.02210
number of training points:  24
Degree 2 e_LOO 0.0019065341772600804 Pf 0.01942 CoV 0.02250
number of training points:  25
Degree 2 e_LO

Degree 1 e_LOO 0.061183576225642015 Pf 0.00428 CoV 0.04820
number of training points:  13
Degree 1 e_LOO 0.011553262064014209 Pf 0.00693 CoV 0.03790
number of training points:  14
Degree 1 e_LOO 0.013872908156748801 Pf 0.00289 CoV 0.05870
number of training points:  15
Degree 2 e_LOO 0.00692890783817889 Pf 0.03125 CoV 0.01760
number of training points:  16
Degree 2 e_LOO 5.007238715341128e-05 Pf 0.02899 CoV 0.01830
number of training points:  17
Degree 2 e_LOO 8.506030819764121e-05 Pf 0.03215 CoV 0.01740
number of training points:  18
Degree 1 e_LOO 0.001419105475108919 Pf 0.00958 CoV 0.03220
number of training points:  19
Degree 1 e_LOO 0.0009878595992119153 Pf 0.00791 CoV 0.03540
number of training points:  20
Degree 1 e_LOO 0.0007891279898501379 Pf 0.00734 CoV 0.03680
number of training points:  21
Degree 2 e_LOO 8.43394046810027e-05 Pf 0.00814 CoV 0.03490
number of training points:  22
Degree 2 e_LOO 0.0005149649338061688 Pf 0.00674 CoV 0.03840
number of training points:  23
Degree

# Active training - U - LOO - 1 LS

In [290]:
number_experiments = 15
number_active_points = 40

results_file = '_AT_U_Loo_' + str(number_active_points)   # index for results files

# to select the best model (length and order) 
# kernel hyperparameters------------------------------
v = 5/2
#truncation term-------------------------------------
p_max = 5  #for each variable → same truncation , degree of expansion
        
for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '------------------------------------------------------------' )
    
    ActiveTrain = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #initial design of experiments (LHS) --------------------------------

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    mean_loo = np.zeros(len(xn))
    var_loo = np.zeros(len(xn))

    x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
    x2mean, x2sigma = 2.5 , 1.0  # normal distribution 

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn = function(xr)[0]

    for points in range(number_active_points):

        ModelName = 'PCK' + str(len(xn))

        ModelName = PC_Kriging(config)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        eloo_results = np.zeros(p_max-1)
        sumat = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            results = dict()
            bounds = [(lmin, lmax)]
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = PCK1.train (xn, yn, p, theta) 
            mean0, var0 = PCK1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples
        MCS_samples = 100000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  

        # ---------------------------------------------------
        ymc = function(MCinputs)[0]  
        fail_samples = np.sum(ymc < 0 )
        Pf_ref = fail_samples/MCS_samples

        meanMC, varMC = ModelName.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO = np.sum(np.asarray(meanMC) < 0 )
        fail_prob_SUMO = fail_samples_SUMO / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO ) / (fail_prob_SUMO * MCS_samples) )

        # LOO CV errors ###################################################

        for k in range (0, MCS_samples):               #variance enhancement based LOO CV erros around voronoi cells
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+sumat[voro, opt])   
        ####################################################
        
        print('Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf', fail_prob_SUMO , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain[str(len(xn))+'points'] = ModelName , fail_prob_SUMO , cov_pf , np.min(eloo_results), np.min(mse_results)

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), LOOCV.reshape(-1))
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)])w.reshape(-1,2)
        yn = function(xr)[0] #Only one point needs to be evaluated here

        print('number of training points: ', len(xn))
    
    filename = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'
    pickle.dump(ActiveTrain, open(filename, 'wb'))

Experiment:  1 ------------------------------------------------------------


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


Degree 1 e_LOO 0.00016988837566974703 Pf 0.06674 CoV 0.01180
number of training points:  11
Degree 1 e_LOO 9.054516411266028e-05 Pf 0.08575 CoV 0.01030
number of training points:  12
Degree 1 e_LOO 5.100865294683073e-05 Pf 0.0372 CoV 0.01610
number of training points:  13
Degree 1 e_LOO 1.2474324086100414e-05 Pf 0.02471 CoV 0.01990
number of training points:  14
Degree 1 e_LOO 0.005790585287708486 Pf 0.03791 CoV 0.01590
number of training points:  15
Degree 1 e_LOO 0.0013792074652899619 Pf 0.03475 CoV 0.01670
number of training points:  16
Degree 2 e_LOO 7.13098338986533e-05 Pf 0.02367 CoV 0.02030
number of training points:  17
Degree 1 e_LOO 0.0005558999643936644 Pf 0.02303 CoV 0.02060
number of training points:  18
Degree 1 e_LOO 0.0008333248009063205 Pf 0.02779 CoV 0.01870
number of training points:  19
Degree 2 e_LOO 4.7909123079586004e-05 Pf 0.02615 CoV 0.01930
number of training points:  20
Degree 2 e_LOO 0.00022568922411852238 Pf 0.0257 CoV 0.01950
number of training points:  21

C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:129: RuntimeWarning: divide by zero encountered in double_scalars


Degree 2 e_LOO 0.015497381644452274 Pf 0.0 CoV inf
number of training points:  12
Degree 3 e_LOO 0.00020171443552573863 Pf 0.10343 CoV 0.00930
number of training points:  13
Degree 3 e_LOO 0.0013703619378320488 Pf 0.0681 CoV 0.01170
number of training points:  14
Degree 3 e_LOO 6.0318970466081866e-05 Pf 0.06576 CoV 0.01190
number of training points:  15
Degree 3 e_LOO 0.006428881822019729 Pf 0.06005 CoV 0.01250
number of training points:  16
Degree 3 e_LOO 0.004854098798838517 Pf 0.05971 CoV 0.01250
number of training points:  17
Degree 3 e_LOO 0.012480018347100157 Pf 0.05133 CoV 0.01360
number of training points:  18
Degree 1 e_LOO 0.019866166184342827 Pf 0.02752 CoV 0.01880
number of training points:  19
Degree 4 e_LOO 0.0018151018627029418 Pf 0.04811 CoV 0.01410
number of training points:  20
Degree 1 e_LOO 0.0001245444857610885 Pf 0.02515 CoV 0.01970
number of training points:  21
Degree 1 e_LOO 0.00011583828885974115 Pf 0.02521 CoV 0.01970
number of training points:  22
Degree 1 e

Degree 2 e_LOO 0.0002687611908227912 Pf 0.05403 CoV 0.01320
number of training points:  11
Degree 2 e_LOO 0.006710963686685201 Pf 0.01121 CoV 0.02970
number of training points:  12
Degree 2 e_LOO 0.0004541352768892054 Pf 0.01975 CoV 0.02230
number of training points:  13
Degree 2 e_LOO 0.00012918682857019728 Pf 0.0191 CoV 0.02270
number of training points:  14
Degree 2 e_LOO 0.00028476124554014314 Pf 0.02348 CoV 0.02040
number of training points:  15
Degree 2 e_LOO 0.00040356555647705236 Pf 0.0272 CoV 0.01890
number of training points:  16
Degree 2 e_LOO 0.0014822030046251129 Pf 0.01938 CoV 0.02250
number of training points:  17
Degree 1 e_LOO 0.002068797720916333 Pf 0.09097 CoV 0.01000
number of training points:  18
Degree 2 e_LOO 9.902454081637563e-05 Pf 0.01789 CoV 0.02340
number of training points:  19
Degree 2 e_LOO 2.060568134315908e-05 Pf 0.01862 CoV 0.02300
number of training points:  20
Degree 1 e_LOO 0.00013882924595497533 Pf 0.02206 CoV 0.02110
number of training points:  21

Degree 2 e_LOO 1.860288536883112e-06 Pf 0.03094 CoV 0.01770
number of training points:  40
Experiment:  8 ------------------------------------------------------------
Degree 1 e_LOO 0.00012061877825156554 Pf 0.07701 CoV 0.01090
number of training points:  11
Degree 1 e_LOO 1.3062003883140992e-05 Pf 0.03599 CoV 0.01640
number of training points:  12
Degree 1 e_LOO 0.0007270507118539575 Pf 0.00069 CoV 0.12030
number of training points:  13
Degree 1 e_LOO 0.0006814296217617339 Pf 0.00014 CoV 0.26720
number of training points:  14
Degree 1 e_LOO 0.01115110692445752 Pf 0.01074 CoV 0.03030
number of training points:  15
Degree 1 e_LOO 0.0025978721332584223 Pf 0.00826 CoV 0.03470
number of training points:  16
Degree 1 e_LOO 0.001659155234708606 Pf 0.01056 CoV 0.03060
number of training points:  17
Degree 1 e_LOO 3.0064997530230224e-05 Pf 0.01782 CoV 0.02350
number of training points:  18
Degree 1 e_LOO 0.0002976244538051945 Pf 0.0179 CoV 0.02340
number of training points:  19
Degree 1 e_LOO 

Degree 1 e_LOO 0.0028712008969239014 Pf 0.03137 CoV 0.01760
number of training points:  38
Degree 3 e_LOO 0.0006943347618718426 Pf 0.0501 CoV 0.01380
number of training points:  39
Degree 1 e_LOO 0.0011356702586115676 Pf 0.03012 CoV 0.01790
number of training points:  40
Experiment:  11 ------------------------------------------------------------
Degree 2 e_LOO 0.00022419129750216857 Pf 0.0 CoV inf
number of training points:  11
Degree 1 e_LOO 0.0002923513747727694 Pf 0.07841 CoV 0.01080
number of training points:  12
Degree 2 e_LOO 0.006736248428350061 Pf 0.00061 CoV 0.12800
number of training points:  13
Degree 2 e_LOO 0.006558174007720463 Pf 0.01001 CoV 0.03140
number of training points:  14
Degree 1 e_LOO 0.024172738791297214 Pf 0.02307 CoV 0.02060
number of training points:  15
Degree 1 e_LOO 0.017781760514594402 Pf 0.01421 CoV 0.02630
number of training points:  16
Degree 4 e_LOO 0.00537527217294218 Pf 0.09737 CoV 0.00960
number of training points:  17
Degree 2 e_LOO 0.0005559343

Degree 1 e_LOO 4.2876905143927964e-05 Pf 0.02997 CoV 0.01800
number of training points:  37
Degree 2 e_LOO 0.0001192727362018814 Pf 0.03037 CoV 0.01790
number of training points:  38
Degree 2 e_LOO 5.812931530442466e-05 Pf 0.03032 CoV 0.01790
number of training points:  39
Degree 2 e_LOO 4.6709308503535865e-05 Pf 0.03071 CoV 0.01780
number of training points:  40
Experiment:  14 ------------------------------------------------------------
Degree 2 e_LOO 2.3058878637066145e-05 Pf 0.09011 CoV 0.01000
number of training points:  11
Degree 1 e_LOO 0.0002914019474952422 Pf 0.0847 CoV 0.01040
number of training points:  12
Degree 1 e_LOO 0.0007574555929471924 Pf 0.0736 CoV 0.01120
number of training points:  13
Degree 2 e_LOO 0.0007668982367712867 Pf 0.00022 CoV 0.21320
number of training points:  14
Degree 2 e_LOO 0.001004395162074037 Pf 0.00097 CoV 0.10150
number of training points:  15
Degree 2 e_LOO 0.0006515059960013473 Pf 0.00054 CoV 0.13600
number of training points:  16
Degree 2 e_LO

# Active training - EFF - LOO - 1 LS

In [ ]:
number_experiments = 13
number_active_points = 40

results_file = '_AT_EFF_Loo_' + str(number_active_points)   # index for results files

# to select the best model (length and order) 
# kernel hyperparameters------------------------------
v = 5/2
#truncation term-------------------------------------
p_max = 5  #for each variable → same truncation , degree of expansion
        
for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '------------------------------------------------------------' )
    
    ActiveTrain = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #initial design of experiments (LHS) --------------------------------

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    mean_loo = np.zeros(len(xn))
    var_loo = np.zeros(len(xn))

    x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
    x2mean, x2sigma = 2.5 , 1.0  # normal distribution 

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn = function(xr)[0]

    for points in range(number_active_points):

        ModelName = 'PCK' + str(len(xn))

        ModelName = PC_Kriging(config)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        eloo_results = np.zeros(p_max-1)
        sumat = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xr, xr)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            results = dict()
            bounds = [(lmin, lmax)]
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = PCK1.train (xn, yn, p, theta) 
            mean0, var0 = PCK1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples
        MCS_samples = 100000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  

        # ---------------------------------------------------
        ymc = function(MCinputs)[0]  
        fail_samples = np.sum(ymc < 0 )
        Pf_ref = fail_samples/MCS_samples

        meanMC, varMC = ModelName.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO = np.sum(np.asarray(meanMC) < 0 )
        fail_prob_SUMO = fail_samples_SUMO / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO ) / (fail_prob_SUMO * MCS_samples) )

        # LOO CV errors ###################################################

        for k in range (0, MCS_samples):               #variance enhancement based LOO CV erros around voronoi cells
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+sumat[voro, opt])   
        ####################################################
        
        print('Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf', fail_prob_SUMO , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain[str(len(xn))+'points'] = ModelName , fail_prob_SUMO , cov_pf , np.min(eloo_results), np.min(mse_results)

        ### Evaluating new points
                ### Evaluating new points
        eff1 = EFF(meanMC.reshape(-1),LOOCV.reshape(-1) , 0 )  #(mean, variance, target)  
        xr = np.append(xr, MCinputs[np.argmax(eff1)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff1)]).reshape(-1,2)
        
        yn = function(xr)[0] #Only one point needs to be evaluated here

        print('number of training points: ', len(xn))
    
    filename = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'
    pickle.dump(ActiveTrain, open(filename, 'wb'))

# LeaveOneOut CrossValidation

In [ ]:
#Generating PCK models for each reduced design of experiments 
PCK_loo = PC_Kriging(config)    # same 'config' as specified in the original model

mean_loo = np.zeros(n)
var_loo = np.zeros(n)

for i in range (0,n):             
    yn_loo=  np.delete(yn,[i])                              #y_n-i      leaving element i out the observations 
    xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)      #x1r_n-i   leaving element i out the inputs (xr)
    xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)      #x_n-i     leaving element i out the nomalized inputs (xn)

    #training LOO
    modelpar_loo = PCK_loo.train (xn_loo,yn_loo,p,theta)   # same (p, theta) as specified in the original model
     
    #predicting LOO
    mean_loo[i], var_loo[i] = PCK_loo.predict(xn[i].reshape(1,-1))
               
e_loo = (yn - mean_loo)**2              #LOO CV squared errors
sumat = np.divide(e_loo,var_loo)        #vector (e_loo / s_loo) for each observed point

LOOCV=np.zeros(ntest)

for k in range (0,ntest):               #variance enhancement based LOO CV erros around voronoi cells
    voro = VoronoiCell(XR[k],xr)
    
    LOOCV[k]= var_plot[k]*(1+sumat[voro])   

# 2D Plot

In [ ]:
%%time
#predictions over a grid (for plotting)--------------------------------------------------

X1t,X2t = np.meshgrid(np.sort(XN[:,0]), np.sort(XN[:,1])) #grid over normalized points
Xt=np.stack((X1t,X2t),axis=2).reshape(-1,2) #sorting the random sampling (graphing purpose)

X1m,X2m = np.meshgrid(np.sort(XR[:,0]), np.sort(XR[:,1])) #grid over non-normalized points
Xtm=np.stack((X1m,X2m),axis=2).reshape(-1,2) #sorting the random sampling (graphing purpose)

yt=function(Xtm)    #test points
Mtest=yt[0]

response_plot=PCK1.predict(Xt)   #model predictions over (ntest x ntest) points
mean_plot=response_plot[0]
var_plot=response_plot[1]

In [ ]:
fig =plt.subplots(figsize=(12, 8), dpi=80)

levels = 7
cs = plt.contourf(np.sort(XR[:,0]), np.sort(XR[:,1]), Mtest.reshape(ntest,-1), levels)
cbar=plt.colorbar(cs)

csm = plt.contour(np.sort(XR[:,0]), np.sort(XR[:,1]), mean_plot.reshape(ntest,-1) ,levels, colors='k')
plt.clabel(csm, fontsize=9, inline=1)

plt.scatter(xr[:,0],xr[:,1],color='red')

#-------------------------------------------------------------------------
# # # to check Voronoi cells influence vs observed points
point = 44    #point index from MCinputs(any test array) 

argum = VoronoiCell(XR[point],xr)

print(XR[point], xr[argum], argum)

plt.scatter(xr[argum,0],xr[argum,1],color='blue',marker = "x", s = 200)  #observations
plt.scatter(XR[point,0],XR[point,1],color='blue',marker = "*", s = 200)  #test point
#-------------------------------------------------------------------------

cs.changed()